In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))

PYSPARK_PYTHON = "/root/miniconda3/envs/test/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

from offline import SparkSessionBase
import pyhdfs
import time


class UpdateUserProfile(SparkSessionBase):
    """离线相关处理程序
    """
    SPARK_APP_NAME = "updateUser"
    ENABLE_HIVE_SUPPORT = True

    SPARK_EXECUTOR_MEMORY = "4g"

    def __init__(self):

        self.spark = self._create_spark_session()

uup = UpdateUserProfile()

#### 读取部分用户行为日志数据

In [4]:
uup.spark.sql("use profile")

DataFrame[]

In [5]:
user_action = uup.spark.sql("select actionTime,readTime,channelId,param.action,param.userId,param.articleId,param.algorithmCombine from user_action where dt<='2019-03-15'")

In [6]:
user_action.show()

+-------------------+--------+---------+--------+------+--------------------+----------------+
|         actionTime|readTime|channelId|  action|userId|           articleId|algorithmCombine|
+-------------------+--------+---------+--------+------+--------------------+----------------+
|2019-03-07 10:05:06|        |        0|exposure|     2|             [15196]|              C2|
|2019-03-07 10:05:08|        |       18|   click|     2|               15196|              C2|
|2019-03-07 10:05:06|        |        0|exposure|     2|             [15196]|              C2|
|2019-03-07 10:05:23|   14002|       18|    read|     2|               15196|              C2|
|2019-03-07 10:05:08|        |       18|   click|     2|               15196|              C2|
|2019-03-07 10:05:27|        |       18|   click|     2|               18836|              C2|
|2019-03-07 10:05:23|   14002|       18|    read|     2|               15196|              C2|
|2019-03-07 10:05:29|     835|       18|    read| 

#### 用户日志处理 

In [7]:
def _compute(row):
    _list = []
    if row.action == 'exposure':
        for article_id in eval(row.articleId):
            _list.append([row.userId,row.actionTime,article_id,row.channelId,False,False,False,True,row.readTime])
        return _list
    else:
        class Temp(object):
            shared = False
            clicked = False
            collected = False
            read_time = ""
            
        _tp = Temp()
        if row.action == 'click':
            _tp.clicked = True
        elif row.action == 'share':
            _tp.share = True
        elif row.action == 'collect':
            _tp.collect = True
        elif row.action == 'read':
            _tp.read = True
        else:
            pass
        
        _list.append([row.userId,row.actionTime,int(row.articleId),row.channelId,_tp.shared,_tp.clicked,_tp.collected,True,row.readTime])
        return _list
            

In [8]:
_res = user_action.rdd.flatMap(_compute)
user_action_basic = _res.toDF(["user_id","action_time","article_id","channel_id","shared","clicked","collected","exposure","read_time"])
user_action_basic.show()

+-------+-------------------+----------+----------+------+-------+---------+--------+---------+
|user_id|        action_time|article_id|channel_id|shared|clicked|collected|exposure|read_time|
+-------+-------------------+----------+----------+------+-------+---------+--------+---------+
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      1|2019-03-15 10:46:02|     18743|         0| false|  false|    false|    true|         |
|      2|2019-03-15 10:59:04|     13890|

#### 与历史数据合并一起写入user_article_basic表

In [9]:
old = uup.spark.sql("select * from user_article_basic")

In [10]:
new = old.unionAll(user_action_basic)

In [11]:
new.registerTempTable('temptable')
uup.spark.sql(
        "insert overwrite table user_article_basic select user_id, max(action_time) as action_time, "
        "article_id, max(channel_id) as channel_id, max(shared) as shared, max(clicked) as clicked, "
        "max(collected) as collected, max(exposure) as exposure, max(read_time) as read_time from temptable "
        "group by user_id, article_id")

DataFrame[]

#### 用户画像关键词获取以及权重计算

In [12]:
# 1、读取user_article_basic表
user_basic = uup.spark.sql("select * from user_article_basic").drop("channel_id")
user_basic.show()

+-------------------+-------------------+----------+------+-------+---------+--------+---------+
|            user_id|        action_time|article_id|shared|clicked|collected|exposure|read_time|
+-------------------+-------------------+----------+------+-------+---------+--------+---------+
|1105045287866466304|2019-03-11 18:13:45|     14225| false|  false|    false|    true|         |
|1106476833370537984|2019-03-15 16:46:50|     14208| false|  false|    false|    true|         |
|                  1|2019-03-07 16:57:34|     44386| false|   true|    false|    true|    17850|
|                  1|2019-03-11 18:13:11|     44696| false|  false|    false|    true|         |
|                 10|2019-03-06 10:06:15|     43907| false|  false|    false|    true|         |
|1105093883106164736|2019-03-12 11:35:01|    140357| false|  false|    false|    true|         |
|1106473203766657024|2019-03-15 16:32:24|     16005| false|  false|    false|    true|         |
|                 33|2019-03-0

In [13]:
# 因为测试数据的article_profile中只有1~10篇文章，与用户行为表的article_id没有交集，因此转换article_id
def change_article_id(row):
    if row.article_id == 14225:
        article_id = 1
    elif row.article_id == 14208:
        article_id = 2
    elif row.article_id == 44386:
        article_id = 3
    elif row.article_id == 44696:
        article_id = 4
    elif row.article_id == 43907:
        article_id = 5
    elif row.article_id == 140357:
        article_id = 6
    elif row.article_id == 16005:
        article_id = 7 
    elif row.article_id == 13570:
        article_id = 8
    elif row.article_id == 17632:
        article_id = 9
    else:
        article_id = 10
    return row.user_id,row.action_time,article_id,row.shared,row.clicked,row.collected,row.exposure,row.read_time

user_basic = user_basic.rdd.map(change_article_id).toDF(["user_id","action_time","article_id","shared","clicked","collected","exposure","read_time"])

user_basic.show()

+-------------------+-------------------+----------+------+-------+---------+--------+---------+
|            user_id|        action_time|article_id|shared|clicked|collected|exposure|read_time|
+-------------------+-------------------+----------+------+-------+---------+--------+---------+
|1105045287866466304|2019-03-11 18:13:45|         1| false|  false|    false|    true|         |
|1106476833370537984|2019-03-15 16:46:50|         2| false|  false|    false|    true|         |
|                  1|2019-03-07 16:57:34|         3| false|   true|    false|    true|    17850|
|                  1|2019-03-11 18:13:11|         4| false|  false|    false|    true|         |
|                 10|2019-03-06 10:06:15|         5| false|  false|    false|    true|         |
|1105093883106164736|2019-03-12 11:35:01|         6| false|  false|    false|    true|         |
|1106473203766657024|2019-03-15 16:32:24|         7| false|  false|    false|    true|         |
|                 33|2019-03-0

In [14]:
# 2.读取文章画像得主题词
uup.spark.sql("use article")
article_topic = uup.spark.sql("select article_id,channel_id,topics from article_profile")
article_topic.show()

+----------+----------+--------------------------+
|article_id|channel_id|                    topics|
+----------+----------+--------------------------+
|         7|        17|  [交易, eth, 合约, con...|
|         6|        17|      [vue, https, &#, ...|
|         9|        17|    [item, 关键, goLink...|
|         5|        17|      [obj, name, let, ...|
|         1|        17|      [props, child, sp...|
|        10|        17|   [WebSocket, 源码, 程...|
|         3|        17|[内存, Array, 数据类型,...|
|         8|        17|  [模式, webpack, 插件,...|
|         2|        17|    [match, 属性, node,...|
|         4|        17|   [bin, nodejs, 数据库...|
+----------+----------+--------------------------+



In [20]:
# 3.合并用户行为表与文章主题词
user_topic = user_basic.join(article_topic,on=['article_id'],how='left')
user_topic.show()

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|                    topics|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------------+
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|      [props, child, sp...|
|         2|1106476833370537984|2019-03-15 16:46:50| false|  false|    false|    true|         |        17|    [match, 属性, node,...|
|         3|                  1|2019-03-07 16:57:34| false|   true|    false|    true|    17850|        17|[内存, Array, 数据类型,...|
|         4|                  1|2019-03-11 18:13:11| false|  false|    false|    true|         |        17|   [bin, nodejs, 数据库...|
|         5|                 10|2019-03-06 10:06:15| false|  false

In [21]:
import pyspark.sql.functions as F
user_topic = user_topic.withColumn("topic",F.explode("topics")).drop("topics")
user_topic.show()

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+-----------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|      topic|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+-----------+
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|      props|
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|      child|
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|       span|
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|       组件|
|         1|1105045287866466304|2019-03-11 18:13:45| false|  false|    false|    true|         |        17|        Vue|
|         1|1105045287866466304|2019-03-11

In [144]:
# 4.计算用户关键词权重
def compute_user_label_weights(partitions):

    weightsOfaction = {
        "read_min": 1,
        "read_middle": 2,
        "collect": 2,
        "share": 3,
        "click": 5
    }
    
    # 导入包
    from datetime import datetime
    import numpy as np
    import happybase
    import json
    
    # 循环每个用户对应每个关键词处理
    for row in partitions:
        
        # 计算时间系数
        t = datetime.now() - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
        alpha = 1 / (np.log(t.days + 1) + 1)
        
        # 判断一下这个关键词对应的操作文章时间大小的权重处理
        if row.read_time  == '':
            read_t = 0
        else:
            read_t = int(row.read_time)
        
        # 阅读时间的行为分数计算出来
        read_score = weightsOfaction['read_middle'] if read_t > 1000 else weightsOfaction['read_min']
        
        # 计算row.topic的权重
        weights = alpha * (row.shared * weightsOfaction['share'] + row.clicked * weightsOfaction['click'] +
                          row.collected * weightsOfaction['collect'] + read_score) 
        
        
    pool = happybase.ConnectionPool(size=3, host='hadoop1')

    with pool.connection() as conn:
        table = conn.table('user_profile')
        table.put('user:{}'.format(row.user_id).encode(),
                  {'partial:{}:{}'.format(row.channel_id, row.topic).encode(): json.dumps(weights).encode()})
        
user_topic.foreachPartition(compute_user_label_weights)

#### 将用户的基本信息也添加到用户画像

In [23]:
uup.spark.sql("use toutiao")
user_basic = uup.spark.sql("select user_id,gender,birthday from user_profile")

def _update_user_basic(partition):
    from datetime import datetime
    import json
    import happybase

    pool = happybase.ConnectionPool(size=10,host='hadoop1')
    for row in partition:

        from datetime import date
        age = 0
        if row.birthday != 'null':
            born = datetime.strptime(row.birthday,'%Y-%m-%d')
            today = date.today()
            age = today.year - born.year - ((today.month,today.month) < (born.month,born.day))

        with pool.connection() as conn:
            table = conn.table('user_profile')
            table.put('user:{}'.format(row.user_id).encode(),
                     {'basic:gender'.encode():json.dumps(row.gender).encode()})
            table.put('user:{}'.format(row.user_id).encode(),
                     {'basic:birthday'.encode():json.dumps(age).encode()})
            conn.close()

user_basic.foreachPartition(_update_user_basic)